여기에 블렌딩 관련 작업

In [3]:
# from desktop w/ gpu
import tensorflow as tf
import numpy as np
import cv2
import time
import os
from enum import Enum
import keras_cv
import keras
import matplotlib.pyplot as plt
from utils import *

print(keras_cv.__version__)
print(keras.__version__)

print(tf.config.list_physical_devices('GPU'))
print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())
print(tf.__version__)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices() )
print(cv2.getBuildInformation())

%load_ext autoreload
%autoreload 2



2025-05-02 17:23:25.376914: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 17:23:25.585493: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-02 17:23:25.585536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-02 17:23:25.618393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-02 17:23:25.691417: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend
0.9.0
2.15.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
4.12.0-dev
1
2.15.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4776740988916543827
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10177478656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12395946626378228467
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]

General configuration for OpenCV 4.12.0-dev =====================================
  Version control:               4.11.0-356-g325e59bd4c

  Extra modules:
    Location (extra):            /home/primarina314/opencv_contrib/modules
    Version control (extra):     4.11.0-42-gacd145f4

  Platform:
    Timestamp:                   2025-04-28T09:41:11Z
    Host:                        Linux 5.15.167.4-microsoft-standard-WSL2 x86_64
    CMake:                 

2025-05-02 17:23:27.749328: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-02 17:23:27.791491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-02 17:23:27.791527: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-02 17:23:27.806267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-02 17:23:27.806324: I external/local_xla/xla/stream_executor

In [1]:
# from laptop w/o gpu
import numpy as np
import cv2
import time
import os
from enum import Enum
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from utils import *

print(cv2.__version__)

%load_ext autoreload
%autoreload 2

4.12.0-dev


# 경계 블렌딩

우선 아래 방법 적용하지 않은 상태로 구현

- 원본 파트에서 경계에서 떨어진 거리에 따라 블렌딩 비율 다르게
- 경계에 가까울수록 downscaled->upscaled 이미지 비율을 높게 잡고
- 경계에서 멀어질수록 original part 이미지 비율을 높게 설정하여 블렌딩
- 일정 거리 이상으로 멀어지면 100% 원본파트
- 선형, sigmoid 등 함수로 interpolation

In [2]:
class blend_mode(Enum):
    alpha = 0
    poisson = 1
    laplacian = 2
    grad = 3
    dl = 4

def blending_two_images(src, target, mask, mode = blend_mode.poisson, center = (0, 0)):
    match mode:
        case blend_mode.alpha:
            result = cv2.addWeighted(src, 0.7, target, 0.3, 0, mask=mask)
        case blend_mode.poisson:
            result = cv2.seamlessClone(
                src, target, mask, center, cv2.MIXED_CLONE
            )
        case blend_mode.laplacian:
            # laplacian dist
            return
        case blend_mode.grad:
            # gradient
            return
        case blend_mode.dl:
            # dl model
            return

    return result

In [5]:
img_path = 'Lenna_(test_image).png'

# down->upscaled 된거 path
# original part path


In [5]:
mrs3_compress('Lenna_(test_image).png', 'qwergfhfghgfhgfg', 4, ROI_mode.Polygon)


original file: 473831
downscaled filesize: 35816
roi filesize: 27504
config filesize: 97
compression ratio: 0.1338388581582885


In [6]:
mrs3_restore('qwergfhfghgfhgfg', MRS3_mode.edsr, 'qwergfhfghgfhgfg/restored.png')